# **EDA for the target data - part 1**

## 0 - Setting up

### 0.1 - Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import matplotlib.gridspec as gridspec
sns.set() # set seaborn as default style

from matplotlib.ticker import PercentFormatter
plt.rcParams.update({ "figure.figsize" : (8, 5),"axes.facecolor" : "white", "axes.edgecolor":  "black"})
plt.rcParams["figure.facecolor"]= "w"
pd.plotting.register_matplotlib_converters()
pd.set_option('display.float_format', lambda x: '%.3f' % x)

#for dealing with outliers
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest

#for time
import time


#miscellania
import scipy.stats as ss
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings("ignore")


RSEED = 42

### 0.2 - Functions set up by the user

#### Cramers correlation between two categorical variables

In [ ]:
def cramers_corrected_stat(df,cat_col1,cat_col2):
    """
    This function spits out corrected Cramer's correlation statistic
    between two categorical columns of a dataframe 
    """
    crosstab = pd.crosstab(df[cat_col1],df[cat_col2])
    chi_sqr = ss.chi2_contingency(crosstab)[0]
    n = crosstab.sum().sum()
    r,k = crosstab.shape
    phi_sqr_corr = max(0, chi_sqr/n - ((k-1)*(r-1))/(n-1))    
    r_corr = r - ((r-1)**2)/(n-1)
    k_corr = k - ((k-1)**2)/(n-1)
    
    result = np.sqrt(phi_sqr_corr / min( (k_corr-1), (r_corr-1)))
    return round(result,3)

#### Anova p-value for test of no correlation between a numerical and a categorical variables

In [ ]:
def anova_pvalue(df,cat_col,num_col):
    """
    This function returns the anova p-value (probability of no correlation) 
    between a categorical column and a numerical column of a dataframe
    """
    CategoryGroupLists = df.groupby(cat_col)[num_col].apply(list)
    AnovaResults = ss.f_oneway(*CategoryGroupLists)
    p_value = round(AnovaResults[1],6)
    return p_value

### Isolation forest test

In [ ]:
def isolation_forest_test(series, contamination_rate = 0.1, random_state = 0, n_estimators = 100):
    """
    Function that takes a time series and perform an isolation forest test
    and returns an boolean array with True/False for is/ is not an outlier
    """
    
    #put the pandas series into the numpy array format
    array = series.array.reshape(-1, 1)

    # fit the isolation forest model
    isolation_forest = IsolationForest(n_estimators = n_estimators, 
                                       contamination=contamination_rate, 
                                       random_state = random_state, n_jobs = -1)
    isolation_forest.fit(array)

    #predict outliers
    is_outlier = isolation_forest.predict(array) == -1

    return is_outlier

#### Creating reports on the time series in a dataframe

In [ ]:
def time_series_reports(data, index, variable, series_id):
    """
    Creates a dictionary wehere the keys are the series id and the values 
    are the corresponding reports on the series in a dataframe. The reports
    contain start and en dates, number of points and missing values.
    """
    
    list_series_ids = sorted(list(set([series_id for series_id in data[series_id]])))
    
    series_reports_dict = {} #dict of series reports
    
    for id_ in list_series_ids: #loop through the series ids
        #slice the datasets into time series
        aux_series = data[data[series_id] == id_].set_index(index)[variable]

        #retrieve info for the report on production
        start_date = aux_series.index[0]
        end_date = aux_series.index[-1]
        n_points = aux_series.size
        nans_series = aux_series[aux_series != aux_series]
    
        #make the dictionary of reports
        series_reports_dict.update({id_ : [start_date, end_date, n_points, nans_series]})
        
    return series_reports_dict

In [ ]:
def rescale_time_series(series_reference, series_to_scale):
    """
    Rescale the time series in a dictionary using as reference
    its counterpart in another dictionary using the MinMax method.
    """

    scaler = MinMaxScaler() #instanciated the MinMax scaler
    scaler.fit(series_reference.array.reshape(-1,1))
    aux_array = scaler.transform(series_to_scale.array.reshape(-1,1)).flatten()
    
    scaled_series = pd.Series(data = aux_array, 
                              index = series_to_scale.index)
    
    return scaled_series

## 1 - Data documentation

- `county` - An ID code for the county.
- `is_business` - Boolean for whether or not the prosumer is a business.
- `product_type` - ID code with the following mapping of codes to contract types: {0: "Combined", 1: "Fixed", 2: "General service", 3: "Spot"}.
- `target` - The consumption or production amount for the relevant segment for the hour. The segments are defined by the county, is_business, and product_type.
- `is_consumption` - Boolean for whether or not this row's target is consumption or production.
- `datetime` - The Estonian time in EET (UTC+2) / EEST (UTC+3). It describes the start of the 1-hour period on which target is given.
- `data_block_id` - All rows sharing the same data_block_id will be available at the same forecast time. This is a function of what information is available when forecasts are actually made, at 11 AM each morning. For example, if the forecast weather data_block_id for predictins made on October 31st is 100 then the historic weather data_block_id for October 31st will be 101 as the historic weather data is only actually available the next day.
- `row_id` - A unique identifier for the row.
- `prediction_unit_id` - A unique identifier for the county, is_business, and product_type combination. New prediction units can appear or disappear in the test set.

## 2 - Loading the data

In [ ]:
# Reading my csv file
train_df = pd.read_csv('../data/train.csv')
train_df.head()

In [ ]:
train_df.tail()

In [ ]:
#put date/time into the datetime type
train_df['datetime'] = pd.to_datetime(train_df['datetime'])

In [ ]:
#display info about the dataset and count non-NaNs values
train_df.info(show_counts=True)

The `target` columns has missing values.

In [ ]:
#count the number of NaNs in target
train_df['target'].isnull().sum()

Count how many timestamps per `prediction_unit_id`.

In [ ]:
train_df.groupby('prediction_unit_id')['datetime'].count()

Clearly some `prediction_unit_id` have either started been recorded later or stopped being recorded earlier.

## 3 - Getting unique `pediction_unit_id` values and make a dictionary

In [ ]:
#get unique predction_unit_id's
pred_uni_ids = sorted(list(set([pred_id for pred_id in train_df['prediction_unit_id']])))
aux_set = set(list(zip(train_df['county'], train_df['product_type'], train_df['is_business'], train_df['prediction_unit_id'])))
aux_keys = []
aux_values = []
for elem in aux_set:
    aux_keys.append(elem[3])
    aux_values.append([elem[0], elem[1], elem[2]])

pred_ids_dict = dict(sorted(dict(zip(aux_keys,aux_values)).items()))

We put that dictionary in the dataframe format and save as a .csv file.

In [ ]:
pred_ids_df = pd.DataFrame(data = pred_ids_dict).transpose()
pred_ids_df.reset_index(inplace = True)
pred_ids_df.rename(columns = {'index' : 'prediction_unit_id', 
                              0 : 'county', 
                              1 : 'product_type', 
                              2 : 'is_business'}, 
                   inplace = True)
pred_ids_df.head()

In [ ]:
pred_ids_df.to_csv('../data/prediction_unit_id_dictionary.csv', index = False)

## 4 - EDA

### 4.1 - Correlations

We first separate the categorical from numerical features. Here only `target` is a numerical feature.

In [ ]:
# Select the specified features from the DataFrame
cat_feats = ['county', 'is_business', 
             'product_type', 'is_consumption', 
             'prediction_unit_id']

target = ['target']

#### Correlations amongst the categorical features

To analyse the correlations amongst the *categorical features*, we employ __[*Cramér's V correlation*](https://en.wikipedia.org/wiki/Cram%C3%A9r%27s_V)__ coefficient.

In [ ]:
# create a dictionary of Cramer's correlation 
cramer_v_corr = dict(
    zip(
        cat_feats,
        [[cramers_corrected_stat(train_df,f1,f2) for f2 in cat_feats] for f1 in cat_feats]
    )
)

In [ ]:
# Cramer correlation heatmap

plt.figure(figsize=(10,10),dpi=100)
sns.heatmap(data=pd.DataFrame(data=cramer_v_corr,index=cat_feats),
            cmap='magma',
            linecolor='white',
            linewidth=1,
            annot=True,
            vmin=-1,
            vmax=1
           );

We see that `is_business` is correlated with `product_type`. This makes sense. From __[Enefit's](https://www.energia.ee/en/ari/elekter/elektrileping-ja-paketid?consumers=large-consumer)__ website, we see that while large consumers (> 300MWh/year) have access to 4 different types of products, small consumers (< 300MWh/year) only have access to two. From the data documentation and the content in that webpage we can make up the following key: 

- (0) Combined -> Fixed and exchange price purchase solution
- (1) Fixed -> Fixed price purchase solution
- (2) General service -> Base energy and exchange price purchase solution
- (3) Spot -> Exchange price purchase solution

For large consumers all the 4 types are relevant, while for small consumers *(1) Fixed* and *(3) Spot* are relevant. This makes sense with the distribution of `product_type` for `is_consumption` = 0, 1 in the EDA for the `client` dataset.

Moreover, we also see that `county` shows correlations with `is_business` and `product_type`. This makes sense, if we think in terms of demographics. Counties with large population, as Harju, Tartu and Ida-Viru, are more prone to have larger business and thus allow for product types more often thanin counties with smaller population.

#### Correlations amongst the categorical features and the target

To analyse the correlations amongst the categorical features and target, we now employ the __[*ANOVA p-value*](https://en.wikipedia.org/wiki/Analysis_of_variance)__ method. This method computes the probablility of the categoral and numerical values to be correlated.

In [ ]:
# create a dictionary of anova p-value

anova_pvalue_dict = dict(
    zip(
        cat_feats,
        [[anova_pvalue(train_df.dropna(),f1,f2) for f2 in target] for f1 in cat_feats]
    )
)

In [ ]:
# heatmap of anova p-values

plt.figure(figsize=(1,8),dpi=100)
sns.heatmap(data=pd.DataFrame(data=anova_pvalue_dict,index=target).T,
            cmap='viridis',
            linecolor='white',
            linewidth=1,
            annot=True,
            vmin=0,
            vmax=1
           );

It seems that we cannot rule out correlations between the categorical features and the target.

#### Production/consumption correlations

In [ ]:
aux_df = train_df[train_df['is_consumption']==0].merge(train_df[train_df['is_consumption']==1], 
                                                       on = ['datetime','prediction_unit_id', 
                                                             'product_type', 'is_business', 'county'], 
                                                       how = 'inner', 
                                                      suffixes = ('_production', '_consumption'))

In [ ]:
# Pearson correlation heatmap

#plt.figure(figsize=(25,10),dpi=100)
plt.title('Production-consumption correlation')
sns.heatmap(data=aux_df[['target_production', 'target_consumption']].corr(method = 'pearson', 
                                         numeric_only = True), 
            cmap='coolwarm', linecolor='white', linewidth=1, 
            annot=True, vmin=-1, vmax=1)
plt.show()

In [ ]:
#plt.figure(figsize=(25,10),dpi=100)
fig, axes = plt.subplots(1, 2, figsize=(14,6), dpi=100)
gs = gridspec.GridSpec(1,2)
for i in [0,1]:
    axes[i].set_title(f'Production-consumption correlations for is_business = {i}')
    sns.heatmap(data=aux_df[aux_df['is_business'] == i][['target_production', 'target_consumption']].corr(method = 'pearson', 
                                         numeric_only = True), 
            cmap='coolwarm', linecolor='white', linewidth=1, 
            annot=True, vmin=-1, vmax=1, ax=axes[i])

It is interesting to notice that the Pearson correlation in the case of business is much higher than in the case for non-business. There can be different reasons for that, such as

- energy independence and cost savings;
- net metering and excess generation;
- environmental initiatives and corporate social responsibility;
- governament incentives and policies;
- operational patterns and daylight hours;
- scale of solar intallations;
- economic trends.

In [ ]:

#plt.figure(figsize=(25,10),dpi=100)
fig, axes = plt.subplots(2, 2, figsize=(14,14), dpi=100)
gs = gridspec.GridSpec(2,2)
for i in [0,1,2,3]:
    axes[int(i/2), int(i%2)].set_title(f'Production-consumption correlations for product_type = {i}')
    sns.heatmap(data=aux_df[aux_df['product_type'] == i][['target_production', 'target_consumption']].corr(method = 'pearson', 
                                         numeric_only = True), 
            cmap='coolwarm', linecolor='white', linewidth=1, 
            annot=True, vmin=-1, vmax=1, ax=axes[int(i/2), int(i%2)])

We see a higher, negative correlation for `product_type` = 0, 2, which are the product types also share by the small consumers. The reason for the negative correlation can be seasonality: low production and higher consumption during winter months and higher production and lower consumption during the summer months.

In [ ]:
#plt.figure(figsize=(25,10),dpi=100)
fig, axes = plt.subplots(8, 2, figsize=(12,46), dpi=100)
gs = gridspec.GridSpec(8,2)
for i in range(16):
    axes[int(i/2), int(i%2)].set_title(f'Production-consumption correlations for county = {i}')
    sns.heatmap(data=aux_df[aux_df['county'] == i][['target_production', 'target_consumption']].corr(method = 'pearson', 
                                         numeric_only = True), 
            cmap='coolwarm', linecolor='white', linewidth=1, 
            annot=True, vmin=-1, vmax=1, ax=axes[int(i/2), int(i%2)])

When analysing the production-consumption correlation for each county, we notice that the higher correlation are also negative. This could be explain by seasonal effects. Higher correlations are found in counties with low population, thus probably made of many small consumer which would be more prone to seasonal effects.

### 4.2 - `target` distribution

We now take a look at the distribution of the `target`. We will first split the dataframe w.r.t. production and consumption.

In [ ]:
cols = ['datetime', 'target', 'prediction_unit_id']

pred_uni_ids = sorted(list(set([pred_id for pred_id in train_df['prediction_unit_id']])))

cond_prod = train_df['is_consumption'] == 0
cond_cons = train_df['is_consumption'] == 1

prod_df = train_df[cond_prod][cols].copy()
cons_df = train_df[cond_cons][cols].copy()

In [ ]:
plt.figure(figsize=(8, 3), dpi = 100)
sns.histplot(cons_df, x = 'target')
plt.title('Distribution of target for consumption')
plt.xlim(0,2000)
plt.ylim(0,30000)
plt.xlabel('target')
plt.ylabel('frequency')
plt.show()

In [ ]:
plt.figure(figsize=(8, 3), dpi = 100)
sns.histplot(prod_df['target'])
plt.title('Distribution of target for production')
plt.xlim(0,2000)
plt.ylim(0,1500)
plt.xlabel('target')
plt.ylabel('frequency')
plt.show()

We now display the distribution of zero and non-zero values.

In [ ]:
non_zeros_target = cons_df['target'] != 0.
plt.figure(figsize=(8, 3), dpi = 100)
sns.histplot(non_zeros_target)
plt.title('Distribution of target for consumption')
plt.xlabel('is_target_non_zero')
plt.ylabel('frequency')
plt.show()

In [ ]:
print(f'Fraction of non-zero values in consumption: {non_zeros_target.sum()/len(non_zeros_target)}')

In [ ]:
non_zeros_target = prod_df['target'] != 0.
plt.figure(figsize=(8, 3), dpi = 100)
sns.histplot(non_zeros_target)
plt.title('Distribution of target for consumption')
plt.xlabel('is_target_non_zero')
plt.ylabel('frequency')
plt.show()

In [ ]:
print(f'Fraction of non-zero values in consumption: {non_zeros_target.sum()/len(non_zeros_target)}')

The conclusion here is that the distribution for `target` is highly skewed. Moreover, in the case of production, there is an sizeble fraction of zeros in the sample. The skewness of the distributions is a problem for regressions models, which asume a normal distribution for the target variable.

The distribution of the `target` values are skewed towards the small values, having a tail for large values. A way to bring large values closer is to use a logarithmic scale. In our case it is convenient to use the transformation $x \to \log (1 + x)$, which will bring larger values closer to small values but will keep zero fixed.   

In [ ]:
plt.figure(figsize=(8, 3), dpi = 100)
log1p_target = np.log1p(cons_df['target'])
sns.histplot(log1p_target)
plt.title('Distribution of target for consumption in log scale')
plt.xlabel('log1p_target')
plt.ylabel('frequency')
plt.show()

In [ ]:
plt.figure(figsize=(8, 3), dpi = 100)
log1p_target = np.log1p(prod_df['target'])
sns.histplot(log1p_target)
plt.title('Distribution of target for production in log scale')
plt.xlabel('log1p_target')
plt.ylabel('frequency')
plt.show()

We see this idea works for the consumption, making its distribution more symmetrical, but not for production, due to the high occurance of zeros in the sample.

In [ ]:
for county in range(16):
    sns.histplot(np.log1p(train_df[(train_df['is_business']==1)&(train_df['is_consumption']==1)&(train_df['county']==county)]['target']))

In [ ]:
for county in [0,2,5,7,11]:
    sns.histplot(np.log1p(train_df[(train_df['is_business']==1)&(train_df['is_consumption']==1)&(train_df['county']==county)]['target'])).set_xlim(0,11)

In [ ]:
for county in [1,3,4,6,8,9,10,12,13,14,15]:
    sns.histplot(np.log1p(train_df[(train_df['is_business']==1)&(train_df['is_consumption']==1)&(train_df['county']==county)]['target'])).set_xlim(0,11)

In [ ]:
sns.histplot(np.log1p(train_df[(train_df['is_business']==1)&(train_df['is_consumption']==1)]['target'])).set_xlim(0,11)

In [ ]:
#plt.figure(figsize=(25,10),dpi=100)
fig, axes = plt.subplots(2, 2, figsize=(10,10), dpi=100)
gs = gridspec.GridSpec(2,2)
for i in range(4):
    axes[int(i/2), int(i%2)].set_title(f'Target consumption distribution for product_type = {i}')
    aux_series = np.log1p(aux_df[(aux_df['is_business']==1)&(aux_df['product_type']==i)]['target_consumption'])
    sns.histplot(aux_series, label = f'Contract {i}', kde = True, ax=axes[int(i/2), int(i%2)])
    axes[int(i/2), int(i%2)].set_xlim(0,11)

In [ ]:
fig, axes = plt.subplots(8, 2, figsize=(12,46), dpi=100)
gs = gridspec.GridSpec(8,2)

for i in range(16):
    sns.histplot(np.log1p(aux_df[(aux_df['is_business']==1)&(aux_df['county']==i)]['target_consumption']), ax=axes[int(i/2), int(i%2)])
#    axes[int(i/2), int(i%2)].set_xlim(0,11) 
#    axes[int(i/2), int(i%2)].set_ylim(0, 5000)
    axes[int(i/2), int(i%2)].set_title(f'Target consumption distribution fort County {i}')

### 4.3 - Duplicates

In [ ]:
# Check for duplicate rows
duplicates = train_df.duplicated().sum()
print(duplicates)

# Now, the data is cleaned from missing values and duplicates.

### 4.4 - Missing values

In order to check for missing values in each individual time series, we will pivot the dataframes for production and consumption so eact column consists of an individual time series.

In [ ]:
prod_time_series = prod_df.pivot(index='datetime', columns='prediction_unit_id', values='target')
prod_time_series.rename_axis(None, inplace=True, axis=1)
prod_time_series.rename(columns=lambda x: f"unit_{x}", inplace=True)

In [ ]:
cons_time_series = cons_df.pivot(index='datetime', columns='prediction_unit_id', values='target')
cons_time_series.rename_axis(None, inplace=True, axis=1)
cons_time_series.rename(columns=lambda x: f"unit_{x}", inplace=True)

Next, we creat a series of reports for the production and consumption series for each `production_unit_id`. The report is based on a dictionary containing the starting end ending date/time of the series, its length in term of number of points and a table with the NaNs and corresponding dates, if any.

In [ ]:
cons_series_report = time_series_reports(data = cons_df, index = 'datetime', 
                                       variable = 'target', series_id = 'prediction_unit_id')
prod_series_report = time_series_reports(data = prod_df, index = 'datetime', 
                                       variable = 'target', series_id = 'prediction_unit_id')

#### Report for consumption

In [ ]:
print('REPORT FOR CONSUMPTION TIME SERIES')
for id in pred_uni_ids:
    elem = cons_series_report.get(id)
    print(15*'-----')
    print(f'Time series #{id}')
    print('Starting date:', elem[0])
    print('End date:', elem[1])
    print('Length:', elem[2])
    print('NaNs')
    print(elem[3],'\n')

#### Report for production

In [ ]:
print('REPORT FOR PRODUCTION TIME SERIES')
for id in pred_uni_ids:
    elem = prod_series_report.get(id)
    print(15*'-----')
    print(f'Time series #{id}')
    print('Starting date:', elem[0])
    print('End date:', elem[1])
    print('Length:', elem[2])
    print('NaNs')
    print(elem[3],'\n')

From these reports we conclude that (1) the missing values corresponds to the start and the end of the summer time in 2021, 2022 and 2023 and that (2) the time series for production and consumption do not have the same length (or duration) in time, with some starting later or ending earlier than others.

### 4.5 - Inputing missing values due to summer time

To impute the missing values we will keep the EET/EEST timezone convention and simply interpolate the time series with the mean of the nearest neighbours.

In [ ]:
for i in range(69):
    prod_time_series[f'unit_{i}'] = 0.5*(prod_time_series[f'unit_{i}'].fillna(method = 'ffill', limit = 1) 
                                         + prod_time_series[f'unit_{i}'].fillna(method = 'bfill', limit = 1))
    
    cons_time_series[f'unit_{i}'] = 0.5*(cons_time_series[f'unit_{i}'].fillna(method = 'ffill', limit = 1) 
                                         + cons_time_series[f'unit_{i}'].fillna(method = 'bfill', limit = 1))

We then update the `prod_df` and `cons_df` dataframes with the cleaned series.

In [ ]:
prod_df = prod_time_series.melt(value_name = 'target', 
                                var_name = 'prediction_unit_id', 
                                ignore_index = False)
prod_df.dropna(inplace = True)
prod_df.reset_index(inplace = True)
prod_df['prediction_unit_id'] = prod_df['prediction_unit_id'].apply(lambda x: int(x.split('_')[1]))

In [ ]:
cons_df = cons_time_series.melt(value_name = 'target', 
                                var_name = 'prediction_unit_id', 
                                ignore_index = False)
cons_df.dropna(inplace = True)
cons_df.reset_index(inplace = True)
cons_df['prediction_unit_id'] = cons_df['prediction_unit_id'].apply(lambda x: int(x.split('_')[1]))

### 4.6 - Plotting the individual consumption and production time series

To plot the time series for each `prediction_unit_id` value it is convenient to first pivot the dataframe for both production and consumption.

In [ ]:
fig, axes = plt.subplots(23, 3, figsize=(12,46), dpi=100)
gs = gridspec.GridSpec(23,3)
for i in range(69):
    prod_time_series[f'unit_{i}'].plot(ax=axes[int(i/3),int(i%3)], use_index=False, label = 'cons', legend = True).set_title(f'unit_{i}')
    cons_time_series[f'unit_{i}'].plot(ax=axes[int(i/3),int(i%3)], use_index=False, label = 'prod', legend = True)
    plt.tight_layout()

From the plots we see that we have outlier in some time series and some of them are only piecewise continuous within the dataset time window. The piecewise ones, such as `unit_68` might have to be treated separately later.

### 4.6 - Outliers

#### Removing outliers for the individual time series

In order to visualise the outliers, we will first rescale the time series with the MinMax method.

In [ ]:
for i in range(69):
    scaler = MinMaxScaler()
    prod_time_series[f'unit_{i}_scaled'] = scaler.fit_transform(prod_time_series[f'unit_{i}'].array.reshape(-1,1))
    cons_time_series[f'unit_{i}_scaled'] = scaler.fit_transform(cons_time_series[f'unit_{i}'].array.reshape(-1,1))

In [ ]:
scaled_cols = [f'unit_{i}_scaled' for i in range(69)]
prod_scaled_df = prod_time_series.melt(value_vars = scaled_cols,
                                       value_name = 'target', 
                                       var_name = 'prediction_unit_id', 
                                       ignore_index = False)
prod_scaled_df.reset_index(inplace = True)
prod_scaled_df['prediction_unit_id'] = prod_scaled_df['prediction_unit_id'].apply(lambda x: int(x.split('_')[1]))

In [ ]:
cons_scaled_df = prod_time_series.melt(value_vars = scaled_cols,
                                       value_name = 'target', 
                                       var_name = 'prediction_unit_id', 
                                       ignore_index = False)
cons_scaled_df.reset_index(inplace = True)
cons_scaled_df['prediction_unit_id'] = cons_scaled_df['prediction_unit_id'].apply(lambda x: int(x.split('_')[1]))

In [ ]:
prod_time_series.drop(columns = scaled_cols, inplace = True)
cons_time_series.drop(columns = scaled_cols, inplace = True)

And finally make a box plot of all the 69 production time series.  

In [ ]:
plt.figure(figsize=(20,10),dpi=100)
plt.title('Box plot for the scaled production time series')
sns.boxplot(data=prod_scaled_df, x = 'prediction_unit_id', y = 'target')
plt.show()

We now repeat the above procedure for the consumption time series. 

In [ ]:
plt.figure(figsize=(20,10),dpi=100)
plt.title('Box plot for the scaled consumption time series')
sns.boxplot(data=cons_scaled_df, x = 'prediction_unit_id', y = 'target')
plt.show()

np.empty()From the above box plots we can clearly see that some of the time series have outliers, see e.g. time series with `prediction_unit_id` 31 and 61 and the gaps in the data distribution. 

Next, we will employ **isolation forest models** to detect the outliers. A relevant parameter is the `contamination_rate` that specifies the proportion of outliers in the dataset. We will choose it to be of order 1/(number of points) in the time series, as from the box plots above they are just a hand full. We first perform the test for the production time series.

In [ ]:
#create an empty dataframe
is_outlier = np.array([], dtype = 'bool')

#loop through the prediction_unit_id's
for i in range(69):

    #set the fraction of outliers
    outlier_fraction = 6./prod_time_series[f'unit_{i}'].size
    aux_series = prod_time_series[f'unit_{i}'].dropna()
    #generate a boolean array with True/False for is/is not outlier
    aux_array = isolation_forest_test(series = aux_series, 
                                      contamination_rate = outlier_fraction,
                                      random_state = RSEED, 
                                      n_estimators = 150)
    
    is_outlier = np.append(is_outlier, aux_array)

outliers_prod_df = prod_df.copy()
outliers_prod_df['is_outlier'] = is_outlier

And now for consumption.

In [ ]:
#create an empty dataframe
is_outlier = np.array([], dtype = 'bool')

#loop through the prediction_unit_id's
for i in range(69):

    #set the fraction of outliers
    outlier_fraction = 7./cons_time_series[f'unit_{i}'].size
    aux_series = cons_time_series[f'unit_{i}'].dropna()
    #generate a boolean array with True/False for is/is not outlier
    aux_array = isolation_forest_test(series = aux_series, 
                                      contamination_rate = outlier_fraction,
                                      random_state = RSEED, 
                                      n_estimators = 150)
    
    is_outlier = np.append(is_outlier, aux_array)

outliers_cons_df = cons_df.copy()
outliers_cons_df['is_outlier'] = is_outlier

Here are the resulting datasets.

In [ ]:
outliers_cons_df.head()

In [ ]:
outliers_prod_df.head()

We can now visually check that the detection test worked.

In [ ]:
#for pred_id in pred_uni_ids:
#    pick_pred_id = outliers_prod_df['prediction_unit_id'] == pred_id
#    plt.title(f'Production time series with prediction_unit_id = {pred_id}')
#    ax = sns.scatterplot(data = outliers_prod_df[pick_pred_id], x = 'datetime', y = 'target', hue = 'is_outlier')
#    display(ax)
#    plt.show()
#    time.sleep(2)
#    clear_output(wait=True)

In [ ]:
#for pred_id in pred_uni_ids:
#    pick_pred_id = outliers_cons_df['prediction_unit_id'] == pred_id
#    plt.title(f'Consumption time series with prediction_unit_id = {pred_id}')
#    ax = sns.scatterplot(data = outliers_cons_df[pick_pred_id], x = 'datetime', y = 'target', hue = 'is_outlier')
#    display(ax)
#    plt.show()
#    time.sleep(2)
#    clear_output(wait=True)

We can now remove the outliers we have detected and then input them using the nearest neighbours mean. 

In [ ]:
outliers_prod_df.loc[outliers_prod_df['is_outlier'], 'target'] = np.nan
outliers_cons_df.loc[outliers_cons_df['is_outlier'], 'target'] = np.nan

In [ ]:
outliers_prod_df['target'] = 0.5*(outliers_prod_df['target'].fillna(method = 'bfill', limit = 1) 
                                  + outliers_prod_df['target'].fillna(method = 'ffill', limit = 1))
outliers_cons_df['target'] = 0.5*(outliers_cons_df['target'].fillna(method = 'bfill', limit = 1) 
                                  + outliers_cons_df['target'].fillna(method = 'ffill', limit = 1))

In [ ]:
prod_series_without_outliers = outliers_prod_df.pivot(index='datetime', 
                                                      columns='prediction_unit_id', 
                                                      values='target')
prod_series_without_outliers.rename_axis(None, inplace=True, axis=1)
prod_series_without_outliers.rename(columns=lambda x: f"unit_{x}", inplace=True)

In [ ]:
cons_series_without_outliers = outliers_cons_df.pivot(index='datetime', 
                                                      columns='prediction_unit_id', 
                                                      values='target')
cons_series_without_outliers.rename_axis(None, inplace=True, axis=1)
cons_series_without_outliers.rename(columns=lambda x: f"unit_{x}", inplace=True)

In [ ]:
cons_series_without_outliers_scaled = cons_series_without_outliers.copy()
prod_series_without_outliers_scaled = prod_series_without_outliers.copy()
for i in range(69):
    cons_series_without_outliers_scaled[f'unit_{i}_scaled'] = rescale_time_series(cons_time_series[f'unit_{i}'], 
                                                              cons_series_without_outliers[f'unit_{i}'])
    prod_series_without_outliers_scaled[f'unit_{i}_scaled'] = rescale_time_series(prod_time_series[f'unit_{i}'], 
                                                              prod_series_without_outliers[f'unit_{i}'])

unscaled_cols = [f'unit_{i}' for i in range(69)]
cons_series_without_outliers_scaled.drop(columns = unscaled_cols, inplace = True)
prod_series_without_outliers_scaled.drop(columns = unscaled_cols, inplace = True)

In [ ]:
cons_without_outliers_scaled_df = cons_series_without_outliers_scaled.melt(value_vars = scaled_cols,
                                       value_name = 'target', 
                                       var_name = 'prediction_unit_id', 
                                       ignore_index = False)
cons_without_outliers_scaled_df.reset_index(inplace = True)
cons_without_outliers_scaled_df['prediction_unit_id'] = cons_without_outliers_scaled_df['prediction_unit_id'].apply(lambda x: int(x.split('_')[1]))

In [ ]:
prod_without_outliers_scaled_df = prod_series_without_outliers_scaled.melt(value_vars = scaled_cols,
                                       value_name = 'target', 
                                       var_name = 'prediction_unit_id', 
                                       ignore_index = False)
prod_without_outliers_scaled_df.reset_index(inplace = True)
prod_without_outliers_scaled_df['prediction_unit_id'] = prod_without_outliers_scaled_df['prediction_unit_id'].apply(lambda x: int(x.split('_')[1]))

In [ ]:
plt.figure(figsize=(20,10),dpi=100)
plt.title('Box plot for the scaled production time series')
sns.boxplot(data=prod_without_outliers_scaled_df, x = 'prediction_unit_id', y = 'target')
plt.show()

In [ ]:
plt.figure(figsize=(20,10),dpi=100)
plt.title('Box plot for the scaled consumption time series')
sns.boxplot(data=cons_without_outliers_scaled_df, x = 'prediction_unit_id', y = 'target')
plt.show()

**NOTE:** even though we have eliminated the outliers in the individual time series at this point in the notebook, we have not saved them in the final file produced here.

#### No outliers for the whole `target` column

To see that there are no outlier when we stack all the production or the consumption time series together, let us make scatter plot of `target` against `datetime` for all the `prediction_unit_id`s.  

In [ ]:
plt.figure(figsize=(20,10),dpi=100)
sns.scatterplot(data = prod_df, x = 'datetime', y = 'target')
plt.title('Production target data')
plt.show()

In [ ]:
plt.figure(figsize=(20,10),dpi=100)
sns.scatterplot(data = cons_df, x = 'datetime', y = 'target')
plt.title('Consumption target data')
plt.show()

From the scatter plots we do not really see outliers for the stacked production and consumption data.

### 4.8 - Updating the dataframe and saving the cleaned file

Now that we have gotten rid of the **missing values** in the time series, we can plug them back into our dataframe `train_df`.

In [ ]:
prod_df['is_consumption'] = 0
cons_df['is_consumption'] = 1

prod_cons_concat = pd.concat([prod_df, cons_df])

train_df_clean = train_df.copy()
train_df_clean

train_df_clean = train_df_clean.merge(prod_cons_concat, on = ['datetime', 'prediction_unit_id', 'is_consumption'], 
                     how = 'inner', suffixes = ('_to_remove', ''))
train_df_clean.drop(columns = 'target_to_remove', inplace = True)

In [ ]:
train_df_clean.info(show_counts=True)

After the imputation, we restore the original ordering of the dataframe using `row_id`. 

In [ ]:
train_df_clean.sort_values(by = 'row_id', inplace = True)
train_df_clean.reset_index(drop = True, inplace = True)

In [ ]:
train_df_clean.head()

In [ ]:
train_df_clean.tail()